In [ ]:
import tensorflow 
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.layers as layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_tuner as kt
from tools import create_split_gen, plot_fitting_history,class_report,find_errors
tensorflow.keras.utils.set_random_seed(812)
tensorflow.config.experimental.enable_op_determinism()

In [ ]:
train_augmentation = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    zoom_range=0.1,
    horizontal_flip=True,
    width_shift_range=0.1,   
    height_shift_range=0.1 )
train_gen = create_split_gen(train_augmentation, path="/home/ocelot/Desktop/CNN_RPS/Archive/train")

val_gen = create_split_gen(ImageDataGenerator(rescale = 1./255), path="/home/ocelot/Desktop/CNN_RPS/Archive/validation")

    

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=1)
def create_model(hp):
    model = tensorflow.keras.models.Sequential([
        layer.InputLayer(shape=(100,150,1)),
        layer.Conv2D(hp.Choice('conv_1', [32,64]), (3,3), padding='same'),
        layer.BatchNormalization(), 
        layer.Activation("relu"),
        layer.Conv2D(hp.Choice('conv_1', [32,64]), (3,3), padding='same'),
        layer.BatchNormalization(), 
        layer.Activation("relu"),
        layer.MaxPooling2D((2,2)),
        layer.Conv2D(hp.Choice('conv_2', [64, 128]), (3,3), padding='same'),
        layer.BatchNormalization(), 
        layer.Activation("relu"),
        layer.Conv2D(hp.Choice('conv_2', [64, 128]), (3,3), padding='same'),
        layer.BatchNormalization(), 
        layer.Activation("relu"),
        layer.MaxPooling2D((2,2)),
        layer.Conv2D(hp.Choice('conv_3', [128, 256]), (3,3), padding='same'),
        layer.BatchNormalization(), 
        layer.Activation("relu"),
        layer.Conv2D(hp.Choice('conv_3', [128, 256]), (3,3), padding='same'),
        layer.BatchNormalization(), 
        layer.Activation("relu"),
        layer.MaxPooling2D((2,2)),
        layer.GlobalAveragePooling2D(),      
        layer.Dense(hp.Choice('num_units', [64, 128, 256]), activation='relu'),
        layer.Dropout(hp.Float("dropout", min_value=0.25, max_value=0.50, step=0.25)),
        layer.Dense(3, activation='softmax')])
    optim = tensorflow.keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log'))
    model.compile(optimizer=optim, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
tuner = kt.RandomSearch(
    create_model, 
    objective='val_accuracy',  
    max_trials=25,  
    executions_per_trial=3,  
    directory='/home/ocelot/Desktop', 
    project_name='cnn_tuning',
    seed=812
)

In [ ]:
result= tuner.search(train_gen,  
             epochs=20,
             validation_data=val_gen)

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
for param, value in best_hps.values.items():
    print(f"  {param}: {value}")

In [ ]:
best_model = create_model(best_hps)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)
history= best_model.fit(train_gen, epochs=50, callbacks=[early_stop,lr_scheduler], validation_data=val_gen)

In [ ]:
plot_fitting_history(history)

In [ ]:
test_gen = create_split_gen(ImageDataGenerator(rescale = 1./255), path ="/home/ocelot/Desktop/CNN_RPS/Archive/test", shuffle=False)

In [ ]:
testLoss, testAccuracy = best_model.evaluate(test_gen)

In [ ]:
pred = best_model.predict(test_gen)
report = class_report(pred,test_gen)

In [ ]:
report["scissors"]

In [ ]:
report["rock"]

In [ ]:
report["paper"]

In [ ]:
find_errors(pred, test_gen.classes, test_gen)

In [ ]:
best_model.save("/home/ocelot/Desktop/terzo_def.keras")